[Home](../../README.md)

### Seoul Bike Sharing

Dataset: https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand

We will wrangle the bike data set previewed in the last Jupyter Notebook.

> [!Note]
> None of these processes are destructive to the source CSV as long as you save the modified data to a new CSV.

#### Load the required dependencies

In [ ]:
# Import frameworks
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

#### Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

* Pandas DataFrames integrate seamlessly with other data science and machine learning libraries in Python, such as NumPy, SciPy, scikit-learn, and Matplotlib. Additionaly, using a DataFrame makes your code more readble and maintainable.

In [ ]:
data_frame = pd.read_csv("2.1.2.SeoulBikeData_Sample_Data.csv")

#### Dealing with null values

Null values during data analysis can cause runtime errors and unexpected results. It is important to identify null values and deal with them appropriately before training a model.

The `isnull().sum()` method call returns the null values in any column.

- Null data was not captured, likely due to the dataset coming from a reputable source (https://archive.ics.uci.edu).
    - Mentioned in source description

In [ ]:
data_frame.isnull().sum()

#### Remove duplicates

Duplicate data can have detrimental effects on your machine learning models and outcomes, such as reducing data diversity and representativeness, which can lead to overfitting or biased models.

The `duplicated().sum()` method call returns the count of duplicate rows in the data frame.

- No duplicates were detected

In [ ]:
data_frame.duplicated().sum()

#### Replace data

We can run a lambda function on a column to modify its values. For a simple example, let’s convert the Sex to lowercase. To run a function over a complete column, we can use the apply method which iterates over each row and modifies the values.

* Replaced upper cases for seasons for better access.

In [ ]:
data_frame['Seasons'] = data_frame['Seasons'].apply(lambda x: x.lower())
data_frame['Seasons'].head()

We can check that there are no data entry errors by the `unique()` method call.

-  No data entry errors were caught.

In [ ]:
data_frame['Seasons'].unique()

In [ ]:
data_frame['Seasons'] = data_frame['Seasons'].apply(lambda season: 'winter' if season.lower() == 'winter' else ('summer' if season.lower() == 'summer' else season))
data_frame['Seasons'].unique()

#### Remove outliers

Outliers can skew your analysis on numerical columns, and it is important to remove them. We can use the 25th and 75th quartile on numerical data, to get the inter-quartile range. This allows us to estimate an acceptable range, and we can then filter out any values outside this range. Mathematically, outliers are values occurring outside 1.5 times the interquartile range (IQR) from the first quartile (Q1) or third quartile (Q3).

- For my data set, bike counts were very skewed towards the lower end. To counteract this, I chose a range of ...

In [ ]:
# Get the inter-quartile range on the salary column
print(data_frame['Count'].describe())
Q1 = data_frame['Count'].quantile(0.25)
Q3 = data_frame['Count'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a Count above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')

In [ ]:
# Filter salaries within the acceptable range
data_frame = data_frame[(data_frame['Count'] >= Q1 - 1.5 * IQR) & (data_frame['Count'] <= Q3 + 1.5 * IQR)]
print(data_frame['Count'].describe())

#### Scaling features to a common range

Scaling the features makes it easier for machine learning algorithms to find the optimal solution, as the different scales of the features do not influence them.

* I left 100 in difference for outliers

In [ ]:
scale_feature = 'Count'

#the minimum value with space for outliers
MIN_COUNT = 150

#the maximum value with space for outliers
MAX_COUNT = 1100

#scale features
data_frame[scale_feature] = [(X - MIN_COUNT) / (MAX_COUNT - MIN_COUNT) for X in data_frame[scale_feature]]

data_frame.describe()

> [!important]
> You need to save the calculations for each dataset you scale for scaling new values for prediction.
> - Calculations were saved for scaling new values for prediction.

#### Save the wrangled data to CSV

In [ ]:
data_frame.to_csv('../2.2.feature_engineering/2.2.1.wrangled_data.csv', index=False)